In [5]:
import cirq
import numpy as np
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.zne import inference
import json

In [2]:
@cirq.transformer
class SubstituteGate:
    """Transformer to substitute `source` gates with `target` in the input circuit."""

    def __init__(self, source, target):
        self._source = source
        self._target = target

    def __call__(self, circuit, *, context=None):
        batch_replace = []
        for i, op in circuit.findall_operations(lambda op: op.gate == self._source):
            batch_replace.append((i, op, self._target.on(*op.qubits)))
        transformed_circuit = circuit.unfreeze(copy=True)
        transformed_circuit.batch_replace(batch_replace)
        return transformed_circuit

substitute_h_inv = SubstituteGate(cirq.H**-1.0,cirq.H)
substitute_cnot_inv = SubstituteGate(cirq.CNOT**-1.0,cirq.CNOT)

In [3]:
def prepare_correlated(env):
    yield cirq.H(env[1])
    yield cirq.CNOT(env[1],env[0])
    yield cirq.H(env[0])
    yield cirq.H(env[1])

def collide(env,sys,theta):
    yield cirq.CNOT(env,sys)
    yield cirq.rz(rads=2*theta).on(sys)
    yield cirq.CNOT(env,sys)

def collision_pattern_correlated(env,sys,n,theta):
    for i in range(n):
        yield from collide(env[1],sys,theta)

def meas_coherence(sys):
    yield cirq.H(sys)
    yield cirq.measure(sys, key = 'answer')

def collision_circuit(env,sys,n,theta):
    yield cirq.H(sys)
    yield from prepare_correlated(env)
    yield from collision_pattern_correlated(env,sys,n,theta)
    yield from meas_coherence(sys)

In [6]:
n = 7
g = 1
tau = np.pi/6
t = np.array(list(range(1,n+1)))*tau
theta = g*tau

def corrfunc(time):
    return (np.cos(time)**2-np.sin(time)**2)/2

sys = cirq.LineQubit(0)
env = cirq.LineQubit.range(1,3)
repcnt = 1024

<h2>Creating circuits:</h2>

In [8]:
scale_factors = [1,3,5,7]

In [9]:
circs_json = []

In [10]:
for scale_factor in scale_factors:
    for i in range(1,n+1):
        circ = cirq.Circuit(collision_circuit(env,sys,i,theta))
        circ_f = fold_gates_at_random(circ,scale_factor)
        circ_t = substitute_cnot_inv(substitute_h_inv(circ_f))
        circs_json.append(cirq.to_json(circ_t))

In [ ]:
with open('collisional_circuits_1.json','x') as file:
    json.dump(circs_json,file)